In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.probability import FreqDist
import string
from wordcloud import WordCloud
from konlpy.tag import Okt
from collections import Counter
import ast

import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus']=False

path = 'c:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font',family=font_name)

In [69]:
cnam = pd.read_csv('data/전처리1 데이터/충남1.csv')
one = pd.read_csv('data/전처리1 데이터/강원1.csv')
gg = pd.read_csv('data/전처리1 데이터/경기1.csv')
ic = pd.read_csv('data/전처리1 데이터/인천1.csv')
cbook = pd.read_csv('data/전처리1 데이터/충북1.csv')
jeju = pd.read_csv('data/전처리1 데이터/제주도1.csv')
seoul = pd.read_csv('data/전처리1 데이터/서울1.csv')
gnam = pd.read_csv('data/전처리1 데이터/경남1.csv')
gbook = pd.read_csv('data/전처리1 데이터/경북1.csv')
daegu = pd.read_csv('data/전처리1 데이터/대구1.csv')
daej = pd.read_csv('data/전처리1 데이터/대전1.csv')
busan = pd.read_csv('data/전처리1 데이터/부산1.csv')
sejong = pd.read_csv('data/전처리1 데이터/세종1.csv')
ulsan = pd.read_csv('data/전처리1 데이터/울산1.csv')
jnam = pd.read_csv('data/전처리1 데이터/전남1.csv')
jbook = pd.read_csv('data/전처리1 데이터/전북1.csv')
gj = pd.read_csv('data/전처리1 데이터/광주1.csv')

## 결측치 처리 및 형태 변경, 정렬

In [70]:
# 1. 딕셔너리에 문자열이 덮혀있어서 제거
def dict(df):
    for i in range(len(df)):
        df.at[i, 'good_keyword_ratio'] = ast.literal_eval(df.at[i, 'good_keyword_ratio'])
        df.at[i, 'services'] = ast.literal_eval(df.at[i, 'services'])
        df.at[i, 'star&reviews'] = ast.literal_eval(df.at[i, 'star&reviews'])
        df.at[i, 'good_keyword_freq'] = ast.literal_eval(df.at[i, 'good_keyword_freq'])
        df.at[i, 'good_keyword'] = ast.literal_eval(df.at[i, 'good_keyword'])
        df.at[i, 'bad_keyword'] = ast.literal_eval(df.at[i, 'bad_keyword'])        
    return df 

In [71]:
# 2. 리뷰 높은순 정렬
def review_sort(df):
    for i in range(len(df)):
        reviews = df.loc[i, 'star&reviews']
        # NaN인 경우 빈 리스트로 처리
        if isinstance(reviews, float) and np.isnan(reviews):
            sorted_reviews = []
        else:
            sorted_reviews = sorted(reviews, key=lambda x: x[0], reverse=True)
        df.loc[i, 'star&reviews'] = str(sorted_reviews)
        df.at[i, 'star&reviews'] = ast.literal_eval(df.at[i, 'star&reviews'])
    return df

In [72]:
# 3. 키워드 빈도수 정리
def count_keyword(df, keywordlist):
    df['good_keyword_freq'] = pd.Series(dtype=str)
    for i in range(len(df)):
        keyword_freq = {}
        for keyword in keywordlist:
            try:
                if df.loc[i, 'good'].count(keyword):
                    freq = df.loc[i,'good'].count(keyword)
                    keyword_freq[keyword] = freq
            except:
                keyword_freq[keyword] = 0
        df.loc[i,'good_keyword_freq'] = str(keyword_freq)

In [73]:
# 'services'에 nan값을 []로 변경
lists = cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj
for idx, df in enumerate(lists):
    lists[idx]['services'] = df['services'].fillna("[]")
cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj = lists

In [74]:
# 지역별 긍정 키워드 TOP10
one_keywordlist = '청결 객실 시설 바다 직원 속초 이용 가격 예약 위치'.split()
cnam_keywordlist = '청결 시설 객실 이용 직원 주변 조식 가격 화장실 주차'.split()
gg_keywordlist = '청결 객실 직원 이용 시설 주변 주차 침대 조식 가격'.split()
ic_keywordlist = '청결 객실 시설 직원 이용 주변 가격 주차 체크 근처'.split()
cbook_keywordlist = '청결 객실 시설 직원 이용 주변 조식 청주 예약 생각'.split()
jeju_keywordlist = '청결 직원 가격 객실 시설 이용 침대 위치 근처 주차'.split()
seoul_keywordlist = '청결 객실 직원 이용 위치 가격 시설 체크 조식 근처'.split()
gnam_keywordlist = '청결 객실 시설 직원 이용 조식 예약 주변 바다 침대'.split()
gbook_keywordlist = '청결 객실 시설 조식 직원 경주 이용 가격 위치 가족'.split()
daegu_keywordlist = '청결 조식 시설 객실 직원 이용 주변 주차 가격 예약'.split()
daej_keywordlist = '청결 객실 가격 조식 주차 이용 직원 시설 위치 주변'.split()
busan_keywordlist = '청결 객실 직원 위치 바다 이용 가격 시설 해운대 오션'.split()
sejong_keywordlist = '청결 시설 여유 주차 식당 상태 주변 청소 침구 주차공간'.split()
ulsan_keywordlist = '청결 객실 주차 시설 위치 직원 조식 가격 침대 이용'.split()
jnam_keywordlist = '청결 객실 시설 직원 여수 조식 가격 침대 이용 바다'.split()
jbook_keywordlist = '청결 직원 객실 시설 전주 조식 마을 이용 침대 위치'.split()
gj_keywordlist = '청결 조식 시설 직원 객실 이용 주차 침대 가격 주변'.split()

In [75]:
# 키워드 빈도수 세기 모든 지역 진행
lists = [cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj]
keyword_lists =[cnam_keywordlist, one_keywordlist, gg_keywordlist, ic_keywordlist, cbook_keywordlist, jeju_keywordlist, seoul_keywordlist, gnam_keywordlist, gbook_keywordlist,daegu_keywordlist, daej_keywordlist, busan_keywordlist, sejong_keywordlist, ulsan_keywordlist,jnam_keywordlist, jbook_keywordlist, gj_keywordlist]

keyword_dict = {}
for idx, df in enumerate(lists):
    count_keyword(lists[idx], keyword_lists[idx])

cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj = lists

In [76]:
# 모든 지역 진행
lists = cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj

for i in lists:
    dict(i)
    review_sort(i)

cnam, one, gg, ic, cbook, jeju, seoul, gnam, gbook, daegu,daej, busan, sejong, ulsan, jnam, jbook, gj = lists

In [77]:
cnam.to_csv('충남1.csv',index=False)
one.to_csv('강원1.csv',index=False)
gg.to_csv('경기1.csv',index=False)
ic.to_csv('인천1.csv',index=False)
cbook.to_csv('충북1.csv',index=False)
jeju.to_csv('제주도1.csv',index=False)
seoul.to_csv('서울1.csv',index=False)
gnam.to_csv('경남1.csv',index=False)
gbook.to_csv('경북1.csv',index=False)
daegu.to_csv('대구1.csv',index=False)
daej.to_csv('대전1.csv',index=False)
busan.to_csv('부산1.csv',index=False)
sejong.to_csv('세종1.csv',index=False)
ulsan.to_csv('울산1.csv',index=False)
jnam.to_csv('전남1.csv',index=False)
jbook.to_csv('전북1.csv',index=False)
gj.to_csv('광주1.csv',index=False)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 키워드를 입력하면 해당 키워드의 상위 n개의 데이터 추출

In [59]:
# TOP10긍정 키워드 비율 딕셔너리에서 키워드 선택하면 그 키워드 topn 정렬 후 추출
def ratio_topn(df,keyword,n):
# 'good_keyword_ratio' 열의 각 행에 있는 딕셔너리를 기반으로 keyword 키의 값을 추출하여 리스트에 저장
    clean_list = []
    for i in range(len(df['good_keyword_ratio'])):
        dict = df.at[i, 'good_keyword_ratio']
        if keyword in dict:
            clean_list.append(dict[keyword])
        else:  # keyword 키가 없으면 기본값으로 0을 추가
            clean_list.append(0)  
    # 데이터프레임을 정렬
    df_sorted = df.iloc[pd.Series(clean_list).sort_values(ascending=False).index][:n]
    return df_sorted

In [61]:
# TOP10긍정 키워드 빈도수 딕셔너리에서 키워드 선택하면 그 키워드 topn 정렬 후 추출
def freq_topn(df,keyword,n):
    clean_list = []
    for i in range(len(df['good_keyword_freq'])):
        dict = df.at[i, 'good_keyword_freq']
        if keyword in dict:
            clean_list.append(dict[keyword])
        else:  # keyword 키가 없으면 기본값으로 0을 추가
            clean_list.append(0)  
    # 데이터프레임을 정렬
    df_sorted = df.iloc[pd.Series(clean_list).sort_values(ascending=False).index][:n]
    return df_sorted

In [62]:
# TOPn 리뷰 추출
def review_topn(df,hotel_name, n):
    for i in range(len(df)):
        reviews = df[df['hotel_name'] == hotel_name]['star&reviews'].tolist()
        sorted_reviews = sorted(reviews, key=lambda x: x[0], reverse=True)
        result = sorted_reviews[0][:n]
    return result

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 호텔 클릭시 -> 호텔명 / 시설정보 / 리뷰 n개 추출

In [63]:
def click_hotel(df,hotel_name,n):
    hotel = df[df['hotel_name'] == hotel_name]['hotel_name'].iloc[0]
    services = df[df['hotel_name'] == hotel_name]['services'].iloc[0]
    review = review_topn(df,hotel_name,n)
    return hotel, services, review

## 키워드 빈도수로 할 경우

In [64]:
one_sor_freq = freq_topn(one,'청결',10)
one_sor_freq['hotel_name'].tolist()

['더 베네치아 스위트',
 '속초 비즈니스 호텔 카멜',
 '센텀마크 호텔 양양',
 '르부르낙산 바이 체스터톤스',
 '강릉씨티호텔',
 '어반스테이 속초등대해변',
 '속초 아이파크스위트 호텔앤레지던스',
 '청초호베스트호텔',
 '어반스테이 속초해변 AB',
 '하이오션 경포']

In [65]:
click_hotel(one_sor_freq,'속초 비즈니스 호텔 카멜',3)

('속초 비즈니스 호텔 카멜',
 ['조식포함',
  '와이파이',
  '욕실용품',
  '금연',
  'TV',
  '엘레베이터',
  '에어컨',
  '냉장고',
  '짐보관가능',
  '객실샤워실',
  '무료주차',
  '카페',
  '드라이기',
  '카드결제',
  '주차장'],
 [[5.0, '3일 전', '깨끗하고 항상 친절합니다'],
  [5.0, '5일 전', '방도 깔끔하고 수압이나 소음 하나도 불편한게 없었습니다!'],
  [5.0, '6일 전', '역시 믿고 보는 숙소~\n짱입니다^^']])

## 키워드 비율로 할 경우

In [66]:
one_sor_ratio = ratio_topn(one,'청결',10)
one_sor_ratio['hotel_name'].tolist()

['속초 비즈니스 호텔 카멜',
 '더 베네치아 스위트',
 '센텀마크 호텔 양양',
 '강릉씨티호텔',
 '동해오션시티 레지던스호텔',
 '청초호베스트호텔',
 '라마다 속초',
 '오션투유리조트 설악비치 호텔&콘도',
 '속초 아이파크스위트 호텔앤레지던스',
 '하이원 그랜드 호텔 메인타워']

In [67]:
hotel, services, review = click_hotel(one_sor_ratio,'속초 비즈니스 호텔 카멜',3)

In [68]:
hotel

'속초 비즈니스 호텔 카멜'

In [19]:
services

['조식포함',
 '와이파이',
 '욕실용품',
 '금연',
 'TV',
 '엘레베이터',
 '에어컨',
 '냉장고',
 '짐보관가능',
 '객실샤워실',
 '무료주차',
 '카페',
 '드라이기',
 '카드결제',
 '주차장']

In [20]:
review

[[5.0, '3일 전', '깨끗하고 항상 친절합니다'],
 [5.0, '5일 전', '방도 깔끔하고 수압이나 소음 하나도 불편한게 없었습니다!'],
 [5.0, '6일 전', '역시 믿고 보는 숙소~\n짱입니다^^']]